### Visualizing the distribution of the observations

### Load the required libraries

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
%pylab inline

import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBSSSS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
#print 'keys from STAT=',STAT.keys()

#read data
filename=data_dir+'/US_Weather_%s.parquet'%file_index

df=sqlContext.read.parquet(filename)
print df.count()
df.show(5)

Populating the interactive namespace from numpy and matplotlib
12741
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     2|[B8 59 00 58 38 5...|2002.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     2|[90 59 10 5A 20 5...|2003.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     1|[E0 55 00 58 E0 5...|2004.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     2|[50 57 90 55 A0 5...|2005.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     7|[A0 56 F0 56 A0 5...|2006.0|SSSBSSSS|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
only showing top 5 rows



### Select data for a particular station and measurement type

In [2]:
sqlContext.registerDataFrameAsTable(df,'weather')
Query1="SELECT * FROM weather\n\tWHERE measurement='%s'"%('TMIN')
print Query1
df_test = sqlContext.sql(Query1)
print df_test.count(),'rows'
df_test.show()
rows_test=df_test.rdd.map(lambda row:(row.elevation,row.station,unpackArray(row['vector'],np.float16))).collect()
#rows_test1=rows_test.map(lambda (v,y):(v,np.hstack(y))).collect()
 #xs: [x[0] for x in xs]

                         #T=np.vstack(rows_test
#T=T/10.  # scaling to make the temperature be in centingrates
#shape(T)


import pandas as pd

df_test=pd.DataFrame(rows_test)

df_test1=df_test[2].apply(pd.Series)/10

# rename 0 to elvation 

del df_test[2]

df_test3=pd.concat([df_test,df_test1],axis=1)

df_test3

SELECT * FROM weather
	WHERE measurement='TMIN'
2172 rows
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|    402.9| 36.4708|-121.1472|       TMIN|USR0000CPNN|     0|[E0 54 F0 56 80 5...|2002.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMIN|USR0000CPNN|     0|[40 4C 80 51 00 5...|2003.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMIN|USR0000CPNN|     1|[30 54 40 4C 00 C...|2004.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMIN|USR0000CPNN|     0|[40 52 E0 50 E0 5...|2005.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMIN|USR0000CPNN|     0|[E0 50 30 54 E0 5...|2006.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMIN|USR0000CPNN|     0|[E0 50 00 4F 00 4...|2007.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMIN|USR0000CP

,0,1,0,1,2,3,4,5,6,7,...,355,356,357,358,359,360,361,362,363,364
0,402.9,USR0000CPNN,7.800781,11.101562,4.398438,1.099609,2.800781,5.601562,5.000000,3.900391,...,0.600098,-1.099609,-1.099609,-0.600098,6.699219,10.601562,5.000000,2.199219,1.099609,4.398438
1,402.9,USR0000CPNN,1.700195,4.398438,5.601562,7.800781,9.398438,4.398438,5.000000,7.800781,...,4.398438,9.398438,7.199219,3.900391,-1.099609,-2.199219,-2.800781,6.699219,5.601562,3.900391
2,402.9,USR0000CPNN,6.699219,1.700195,-0.600098,-3.300781,-1.099609,2.199219,8.296875,7.199219,...,1.099609,-1.099609,0.000000,0.000000,1.099609,7.199219,6.101562,6.101562,7.800781,6.101562
3,402.9,USR0000CPNN,5.000000,3.900391,3.900391,0.000000,1.099609,0.600098,6.699219,7.199219,...,13.296875,11.101562,8.898438,7.800781,7.199219,7.199219,6.101562,5.000000,5.601562,3.900391
4,402.9,USR0000CPNN,3.900391,6.699219,3.900391,4.398438,5.000000,6.101562,7.800781,4.398438,...,3.300781,0.000000,2.800781,4.398438,8.296875,6.101562,0.600098,-2.199219,0.000000,1.099609
5,402.9,USR0000CPNN,3.900391,2.800781,2.800781,3.900391,1.700195,1.099609,7.199219,3.900391,...,-1.700195,1.099609,5.000000,4.398438,2.800781,1.099609,3.300781,3.900391,5.601562,2.800781
6,402.9,USR0000CPNN,0.600098,1.099609,8.296875,7.199219,2.800781,1.700195,2.199219,1.700195,...,3.900391,2.800781,6.101562,3.300781,-0.600098,-1.700195,0.000000,1.700195,4.398438,1.700195
7,402.9,USR0000CPNN,1.099609,3.300781,0.000000,-2.800781,0.600098,4.398438,2.800781,2.199219,...,3.300781,1.099609,1.099609,0.600098,3.900391,5.601562,5.000000,5.601562,6.699219,5.601562
8,402.9,USR0000CPNN,5.601562,5.000000,1.099609,5.000000,3.300781,4.398438,7.199219,7.199219,...,7.800781,3.900391,3.900391,6.101562,1.099609,2.199219,3.300781,3.300781,0.000000,0.600098
9,111.3,USC00045123,3.300781,2.800781,1.700195,2.800781,1.700195,1.700195,1.700195,1.700195,...,7.800781,2.199219,5.601562,3.300781,3.300781,1.099609,0.000000,-2.199219,-1.099609,1.099609


In [3]:
df_test4=df_test3.iloc[:,2:]
result=df_test4.mean(axis=1)
type(result)

pandas.core.series.Series

In [4]:
df_test6=df_test3.iloc[:,:2]

In [5]:
df_test5=result.to_frame(name='Avg TMIN')
df_test5.shape

(2172, 1)

In [6]:
reg_data=pd.concat([df_test6, df_test5], axis=1)

In [7]:
reg_data.columns = ['Elevation', 'Station', 'Avg TMIN']
reg_data

,Elevation,Station,Avg TMIN
0,402.9,USR0000CPNN,7.144531
1,402.9,USR0000CPNN,8.078125
2,402.9,USR0000CPNN,7.539062
3,402.9,USR0000CPNN,7.808594
4,402.9,USR0000CPNN,7.042969
5,402.9,USR0000CPNN,6.859375
6,402.9,USR0000CPNN,7.105469
7,402.9,USR0000CPNN,7.601562
8,402.9,USR0000CPNN,7.984375
9,111.3,USC00045123,6.816406


In [8]:
import statsmodels.api as sm

## Regression

In [9]:
from pandas.stats.api import ols
regression = ols(y=reg_data['Avg TMIN'], x=reg_data['Elevation'])
regression

/Users/mengtingwang/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://www.statsmodels.org/stable/regression.html
  exec(code_obj, self.user_global_ns, self.user_ns)



-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <x> + <intercept>

Number of Observations:         2172
Number of Degrees of Freedom:   2

R-squared:         0.0002
Adj R-squared:    -0.0003

Rmse:              1.6011

F-stat (1, 2170):     0.4405, p-value:     0.5069

Degrees of Freedom: model 1, resid 2170

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
             x    -0.0001     0.0002      -0.66     0.5069    -0.0004     0.0002
     intercept     7.9851     0.0394     202.76     0.0000     7.9079     8.0623
---------------------------------End of Summary---------------------------------

In [19]:
#sc.stop()